<a href="https://colab.research.google.com/github/pattom72/Sign/blob/master/day5_sign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

import os
import datetime

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/matrix_three_road_signs/"

/content/drive/My Drive/Colab Notebooks/matrix_three_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')
signs = pd.read_csv('data/signnames.csv')

In [5]:

x_train, y_train, x_test, y_test = train['features'], train['labels'], test['features'], test['labels']

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((34799, 32, 32, 3), (34799,), (4410, 32, 32, 3), (4410,))

In [0]:

if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, x_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

  model.fit(
      x_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs= params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (x_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

def predict(model_trained, x_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis =1)

  y_pred_prob = model_trained.predict(x_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_model_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape= input_shape ),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),      

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),                

                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(num_classes, activation='softmax')

                    ])

In [9]:

def train_and_predict(model):
  model_trained = train_model(model, x_train, y_train)
  return predict(model_trained, x_test, y_test)

train_and_predict(get_cnn_model_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.6979 - accuracy: 0.2650 - val_loss: 1.2650 - val_accuracy: 0.5927
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 1.0299 - accuracy: 0.6611 - val_loss: 0.2720 - val_accuracy: 0.9211
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.4734 - accuracy: 0.8507 - val_loss: 0.1027 - val_accuracy: 0.9704
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2873 - accuracy: 0.9116 - val_loss: 0.0465 - val_accuracy: 0.9874
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2042 - accuracy: 0.9390 - val_loss: 0.0367 - val_accuracy: 0.9906


0.9630385487528345

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape= input_shape ),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),      

                    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),                

                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_one']),

                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_two']),

                    Dense(num_classes, activation='softmax')

                    ])

In [0]:

def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

  model.fit(
      x_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs= 5,
      verbose = 0
      )
  
  score = model.evaluate(x_test, y_test, verbose =0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [12]:
space = {
    'batch_size' : hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3,0.5),    
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3,0.7)

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

  0%|          | 0/25 [00:00<?, ?it/s, best loss: ?]
database. History logging moved to new session
59
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.49947082041689506, 'dropout_cnn_block_three': 0.4377287103468549, 'dropout_cnn_block_two': 0.33171000746190915, 'dropout_dense_block_one': 0.3021197770884801, 'dropout_dense_block_two': 0.5052330284993147}
accuracy=0.9476190209388733
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3858808496505331, 'dropout_cnn_block_three': 0.46549749615918423, 'dropout_cnn_block_two': 0.4610673646048968, 'dropout_dense_block_one': 0.4417902709232945, 'dropout_dense_block_two': 0.3805801944099469}
accuracy=0.9517006874084473
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3949302785375894, 'dropout_cnn_block_three': 0.4103730807166443, 'dropout_cnn_block_two': 0.4759366339336462, 'dropout_dense_block_one': 0.45116033260061955, 'dropout_dense_block_two': 0.6455682825445036}
accuracy=0.9208616614341736
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3